In [1]:
# from functools import partial
from pathlib import Path
# from typing import Optional, Tuple
# import cv2
# import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
# from sklearn.cluster import KMeans, MiniBatchKMeans
# from sklearn.decomposition import PCA
# from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import extract_utils as utils
from torch.utils.data import Dataset, DataLoader
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from torch import nn
import torchvision

## Extract Features

In [2]:
# images_list="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/lists/images.txt"
# images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/images"
# model_name="dino_vits16"
# batch_size=1
# output_dir="//home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16"
# which_block=-1
# # Output1
# utils.make_output_dir(output_dir)
# # Models
# model_name = model_name.lower()
# model, val_transform, patch_size, num_heads = utils.get_model(model_name)    #patch size= 16 number of heads= 6
# # print("patch size=", patch_size, "number of heads=", num_heads)
# # Add hook
# if 'dino' in model_name or 'mocov3' in model_name:
#     feat_out = {}
#     def hook_fn_forward_qkv(module, input, output):
#         # print("feat_out.keys()", feat_out.keys())
#         feat_out["qkv"] = output
#     model._modules["blocks"][which_block]._modules["attn"]._modules["qkv"].register_forward_hook(hook_fn_forward_qkv)
# else:
#     raise ValueError(model_name)
# # Dataset
# filenames = Path(images_list).read_text().splitlines()
# dataset = utils.ImagesDataset(filenames=filenames, images_root=images_root, transform=val_transform)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=8)
# # print(f'Dataset size: {len(dataset)=}')
# # print(f'Dataloader size: {len(dataloader)=}')
# # Prepare
# # accelerator = Accelerator(fp16=True, cpu=False)
# accelerator = Accelerator(cpu=False)
# # model, dataloader = accelerator.prepare(model, dataloader)
# model = model.to(accelerator.device)
# model.num_features
# # print("model.num_features=", model.num_features)
# # model.get_submodule
# # Process
# pbar = list(tqdm(dataloader, desc='Processing'))
# # Process
# # pbar = list(tqdm(dataloader, desc='Processing'))
# # print("feat_out.keys()=", feat_out.keys())
# for i, (images, files, indices) in enumerate(pbar):
#     output_dict = {}
#     # print("images.shape=", images.shape, "files =", files, "indices", indices)
#
#     # Check if file already exists
#     id = Path(files[0]).stem
#     # print("id=", id)
#     output_file = Path(output_dir) / f'{id}.pth'
#     # if output_file.is_file():
#     #     pbar.write(f'Skipping existing file {str(output_file)}')
#     #     continue
#
#     # Reshape image
#     P = patch_size
#     B, C, H, W = images.shape
#     H_patch, W_patch = H // P, W // P
#     H_pad, W_pad = H_patch * P, W_patch * P
#     T = H_patch * W_patch + 1  # number of tokens, add 1 for [CLS]
#     # images = F.interpolate(images, size=(H_pad, W_pad), mode='bilinear')  # resize image
#     images = images[:, :, :H_pad, :W_pad]
#     images = images.to(accelerator.device)
#     # print("images.shape after padding=", images.shape)
#
#     # Forward and collect features into output dict
#     if 'dino' in model_name or 'mocov3' in model_name:
#         # accelerator.unwrap_model(model).get_intermediate_layers(images)[0].squeeze(0)
#         model.get_intermediate_layers(images)[0].squeeze(0)
#         # print(model.get_intermediate_layers(images)[0])
#         # output_dict['out'] = out
#         # print("feat_out.keys()=", feat_out.keys())
#         output_qkv = feat_out["qkv"].reshape(B, T, 3, num_heads, -1 // num_heads).permute(2, 0, 3, 1, 4)
#         # print("type(output_qkv)", type(output_qkv))
#         # print("output_qkv.shape", output_qkv.shape)    #3, 1, 6, 931, 64]
#         # print("output_qkv[0].shape", output_qkv[0].shape)
#         # print("output_qkv[1].shape", output_qkv[1].shape)
#         # print("output_qkv[2].shape", output_qkv[2].shape)
#         # output_dict['q'] = output_qkv[0].transpose(1, 2).reshape(B, T, -1)[:, 1:, :]
#         output_dict['k'] = output_qkv[1].transpose(1, 2).reshape(B, T, -1)[:, 1:, :]
#         # print("output_dict[k].shape=", output_dict['k'].shape)
#         # output_dict['v'] = output_qkv[2].transpose(1, 2).reshape(B, T, -1)[:, 1:, :]
#     else:
#         raise ValueError(model_name)
#
#     # print("output_dict.items=", output_dict.items())
#     # Metadata
#     output_dict['indices'] = indices[0]
#     output_dict['file'] = files[0]
#     output_dict['id'] = id
#     output_dict['model_name'] = model_name
#     output_dict['patch_size'] = patch_size
#     output_dict['shape'] = (B, C, H, W)
#     output_dict = {k: (v.detach().cpu() if torch.is_tensor(v) else v) for k, v in output_dict.items()}
#     # for k, v in output_dict.items():
#     #     print("k=", k)
#     #     if torch.is_tensor(v):
#     #         print("v.shape", v.shape)
#     #     else:
#     #         print("v=", v)
#     # print("output_dict.keys()", output_dict.keys())
#     # print("output_dict['k'].shape=", output_dict['k'].shape,"output_dict['indices'] =", indices[0],"output_dict['file'] =", files[0],"output_dict['id']=" , id, "output_dict['model_name'] =", model_name," output_dict['shape'] =(", B, C, H, W,")output_dict['patch_size'] =",  patch_size )
#     # Save
#     accelerator.save(output_dict, str(output_file))
#     accelerator.wait_for_everyone()
# print(f'Saved features to {output_dir}')

In [3]:
# output_dict = {k: (v.detach().cpu() if torch.is_tensor(v) else v) for k, v in output_dict.items()}
# print(output_dict.keys())
# for k, v in output_dict.items():
#     print("k=", k)
#     print("v.shape=", v.shape)


## Extract Eigen Vectors

In [4]:
images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/images"
features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16"
output_dir="//home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/eigs_onlysimsiam3_30"
which_matrix= 'laplacian'
which_color_matrix= 'knn'
which_features= 'k'
normalize=True
threshold_at_zero=True
lapnorm= True
K= 5
image_downsample_factor = None
image_color_lambda = 0.0
multiprocessing = 0

## Incorporating SimSiam

In [5]:
# Define a custom dataset class
class Feature_Dataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __getitem__(self, index):
        return self.features[index]

    def __len__(self):
        return len(self.features)

In [6]:
class SimSiam(nn.Module):
    def __init__(self):
        super().__init__()
        self.projection_head = SimSiamProjectionHead(feats.shape[1], 128,feats.shape[1])
        self.prediction_head = SimSiamPredictionHead(feats.shape[1], 128, feats.shape[1])

    def forward(self, x):
        z = self.projection_head(x)
        p = self.prediction_head(z)
        z = z.detach()
        return z, p

In [ ]:
utils.make_output_dir(output_dir)
feat_list=[]
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
for inp in tqdm(inputs):
    index, features_file = inp
    print(index, features_file)
     # Load
    data_dict = torch.load(features_file, map_location='cpu')
    print(data_dict.keys())   #['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape']
    # print("shape=", data_dict['shape'], "k shape", data_dict['k'].shape, "patch_size=", data_dict['patch_size'])
    image_id = data_dict['file'][:-4]
    print(image_id)
    # Load
    output_file = str(Path(output_dir) / f'{image_id}.pth')
    if Path(output_file).is_file():
        print(f'Skipping existing file {str(output_file)}')
        # break
        # return  # skip because already generated

    # Load affinity matrix
    feats = data_dict[which_features].squeeze().cuda()
    # print("Without normalizing, Features Shape is",feats.shape)
    if normalize:
        feats = F.normalize(feats, p=2, dim=-1)
    # print("After normalization, Features Shape",feats.shape)
    # print("which_matrix=", which_matrix)
    # Eigenvectors of affinity matrix
    if which_matrix == 'affinity_torch':
        W = feats @ feats.T
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
            # print("W shape=", W.shape)
        eigenvalues, eigenvectors = torch.eig(W, eigenvectors=True)
        eigenvalues = eigenvalues.cpu()
        eigenvectors = eigenvectors.cpu()
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)


    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity_svd':
        USV = torch.linalg.svd(feats, full_matrices=False)
        eigenvectors = USV[0][:, :K].T.to('cpu', non_blocking=True)
        eigenvalues = USV[1][:K].to('cpu', non_blocking=True)
        print("which matrix=",which_matrix,"eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity':
        # print("Without normalizing, Features Shape is",feats.shape)
        W = (feats @ feats.T)
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
        W = W.cpu().numpy()
        # print("W shape=", W.shape)
        eigenvalues, eigenvectors = eigsh(W, which='LM', k=K)
        eigenvectors = torch.flip(torch.from_numpy(eigenvectors), dims=(-1,)).T
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of matting laplacian matrix
    elif which_matrix in ['matting_laplacian', 'laplacian']:

        # Get sizes
        B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
        if image_downsample_factor is None:
            image_downsample_factor = P
        H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor

        # Upscale features to match the resolution
        if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
            feats = F.interpolate(
                feats.T.reshape(1, -1, H_patch, W_patch),
                size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
            ).reshape(-1, H_pad_lr * W_pad_lr).T

        ### Feature affinities
        # print("Without normalizing, Features Shape is",feats.shape)

        # W_feat = (feats @ feats.T)
        # feat_list.append(feats)
        feat_dataset = Feature_Dataset(feats)
        if feats.shape[0]%2==0:
            features_dataloader = DataLoader(feat_dataset, batch_size=2, shuffle=True)
        else:
            features_dataloader = DataLoader(feat_dataset, batch_size=2, shuffle=True, drop_last=True)
        model_simsiam = SimSiam()
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model_simsiam.to(device)
        criterion = NegativeCosineSimilarity()
        optimizer = torch.optim.SGD(model_simsiam.parameters(), lr=0.06)
        print("Starting Training")
        for epoch in range(30):
            total_loss = 0
            for x0 in features_dataloader:
            # for (x0), _, _ in features_dataloader:
                x0 = x0.unsqueeze(0).to(device)
                # print(x0.shape)
                x1=torchvision.transforms.RandomAffine(0)(x0)
                # print(x1.shape)
                x0 = x0.squeeze(0).to(device)
                # print("x0.shape=", x0.shape)
                x1 = x1.squeeze(0).to(device)
                # print("x1 shape=", x1.shape)

                z0, p0 = model_simsiam(x0)
                z1, p1 = model_simsiam(x1)
                loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
                total_loss += loss.detach()
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            avg_loss = total_loss / len(features_dataloader)
            print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")
        projected_feature=model_simsiam(feats)
        W_feat=torch.matmul(projected_feature[0], projected_feature[0].t())
        # print("W_feat.shape=", W_feat.shape)
        # print("W_feat.shape=", W_feat.shape)
        # W_feat=contrastive_affinity(feats, feats.T)
        if threshold_at_zero:
            W_feat = (W_feat * (W_feat > 0))
        W_feat = W_feat / W_feat.max()  # NOTE: If features are normalized, this naturally does nothing
        # W_feat = W_feat.cpu().numpy()
        W_feat = W_feat.detach().cpu().numpy()
        # print("W_feat shape=",W_feat.shape)

        ### Color affinities
        # If we are fusing with color affinites, then load the image and compute
        if image_color_lambda > 0:

            # Load image
            image_file = str(Path(images_root) / f'{image_id}.jpg')
            image_lr = Image.open(image_file).resize((W_pad_lr, H_pad_lr), Image.BILINEAR)
            image_lr = np.array(image_lr) / 255.

            # Color affinities (of type scipy.sparse.csr_matrix)
            if which_color_matrix == 'knn':
                W_lr = utils.knn_affinity(image_lr / 255)
            elif which_color_matrix == 'rw':
                W_lr = utils.rw_affinity(image_lr / 255)

            # Convert to dense numpy array
            W_color = np.array(W_lr.todense().astype(np.float32))
            # print("W_color shape", W_color.shape)

        else:

            # No color affinity
            W_color = 0

        # Combine
        W_comb = W_feat + W_color * image_color_lambda  # combination
        D_comb = np.array(utils.get_diagonal(W_comb).todense())  # is dense or sparse faster? not sure, should check
        # print("W_comb shape= ", W_comb.shape, "D_comb shape",  D_comb.shape)
        if lapnorm:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM', M=D_comb)
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM', M=D_comb)
        else:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM')
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM')
        eigenvalues, eigenvectors = torch.from_numpy(eigenvalues), torch.from_numpy(eigenvectors.T).float()
    print("eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)
    # Sign ambiguity
    for k in range(eigenvectors.shape[0]):
        if 0.5 < torch.mean((eigenvectors[k] > 0).float()).item() < 1.0:  # reverse segment
            eigenvectors[k] = 0 - eigenvectors[k]

    # Save dict
    output_dict = {'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors}
    torch.save(output_dict, output_file)

  0%|          | 0/1000 [00:00<?, ?it/s]

0 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0001.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0001
Starting Training
epoch: 00, loss: -0.45162
epoch: 01, loss: -0.67399
epoch: 02, loss: -0.67719
epoch: 03, loss: -0.74213
epoch: 04, loss: -0.77876
epoch: 05, loss: -0.80504
epoch: 06, loss: -0.82188
epoch: 07, loss: -0.84144
epoch: 08, loss: -0.85688
epoch: 09, loss: -0.79983
epoch: 10, loss: -0.81472
epoch: 11, loss: -0.84970
epoch: 12, loss: -0.84718
epoch: 13, loss: -0.84820
epoch: 14, loss: -0.84979
epoch: 15, loss: -0.84551
epoch: 16, loss: -0.84140
epoch: 17, loss: -0.85051
epoch: 18, loss: -0.85147
epoch: 19, loss: -0.83465
epoch: 20, loss: -0.83713
epoch: 21, loss: -0.83378
epoch: 22, loss: -0.83909
epoch: 23, loss: -0.82483
epoch: 24, loss: -0.83732
epoch: 25, loss: -0.82397
epoch: 26, loss: -0.81871
epoch: 27, loss: -0.80969
epoch: 28, loss: -0.81660
epoch: 29, loss: -0.80

  0%|          | 1/1000 [00:56<15:43:34, 56.67s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])
1 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0002.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0002
Starting Training
epoch: 00, loss: -0.32226
epoch: 01, loss: -0.73420
epoch: 02, loss: -0.79475
epoch: 03, loss: -0.83437
epoch: 04, loss: -0.88056
epoch: 05, loss: -0.89614
epoch: 06, loss: -0.90430
epoch: 07, loss: -0.91177
epoch: 08, loss: -0.89474
epoch: 09, loss: -0.88087
epoch: 10, loss: -0.89271
epoch: 11, loss: -0.88830
epoch: 12, loss: -0.90635
epoch: 13, loss: -0.89774
epoch: 14, loss: -0.90996
epoch: 15, loss: -0.88151
epoch: 16, loss: -0.85911
epoch: 17, loss: -0.87802
epoch: 18, loss: -0.88034
epoch: 19, loss: -0.90048
epoch: 20, loss: -0.89974
epoch: 21, loss: -0.89871
epoch: 22, loss: -0.84241
epoch: 23, loss: -0.84683
epoch: 24, loss: -0.86693
epoch: 25, loss: -0.84361
epoch: 26, loss: -0.84034


  0%|          | 2/1000 [01:51<15:27:08, 55.74s/it]

...

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])
156 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0157.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0157
Starting Training
epoch: 00, loss: -0.40790
epoch: 01, loss: -0.64731
epoch: 02, loss: -0.69252
epoch: 03, loss: -0.74237
epoch: 04, loss: -0.79296
epoch: 05, loss: -0.81864
epoch: 06, loss: -0.81250
epoch: 07, loss: -0.82314
epoch: 08, loss: -0.82917
epoch: 09, loss: -0.84311
epoch: 10, loss: -0.85201
epoch: 11, loss: -0.83851
epoch: 12, loss: -0.82610
epoch: 13, loss: -0.83316
epoch: 14, loss: -0.83396
epoch: 15, loss: -0.84896
epoch: 16, loss: -0.84365
epoch: 17, loss: -0.85709
epoch: 18, loss: -0.85295
epoch: 19, loss: -0.85959
epoch: 20, loss: -0.85753
epoch: 21, loss: -0.84971
epoch: 22, loss: -0.85261
epoch: 23, loss: -0.86334
epoch: 24, loss: -0.86664
epoch: 25, loss: -0.85293
epoch: 26, loss: -0.8393

 16%|█▌        | 157/1000 [2:13:43<12:44:38, 54.42s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 375])
157 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0158.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0158
Starting Training
epoch: 00, loss: -0.63655
epoch: 01, loss: -0.94155
epoch: 02, loss: -0.95834
epoch: 03, loss: -0.95917
epoch: 04, loss: -0.94258
epoch: 05, loss: -0.92529
epoch: 06, loss: -0.92236
epoch: 07, loss: -0.93218
epoch: 08, loss: -0.93020
epoch: 09, loss: -0.93047
epoch: 10, loss: -0.91467
epoch: 11, loss: -0.91600
epoch: 12, loss: -0.90964
epoch: 13, loss: -0.90786
epoch: 14, loss: -0.91021
epoch: 15, loss: -0.90594
epoch: 16, loss: -0.91942
epoch: 17, loss: -0.90515
epoch: 18, loss: -0.90828
epoch: 19, loss: -0.89950
epoch: 20, loss: -0.88046
epoch: 21, loss: -0.87038
epoch: 22, loss: -0.85254
epoch: 23, loss: -0.85582
epoch: 24, loss: -0.85131
epoch: 25, loss: -0.85451
epoch: 26, loss: -0.8424

 16%|█▌        | 158/1000 [2:14:22<11:39:32, 49.85s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 300])
158 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0159.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0159
Starting Training
epoch: 00, loss: -0.72895
epoch: 01, loss: -0.93541
epoch: 02, loss: -0.92860
epoch: 03, loss: -0.92573
epoch: 04, loss: -0.89530
epoch: 05, loss: -0.88978
epoch: 06, loss: -0.88508
epoch: 07, loss: -0.88771
epoch: 08, loss: -0.86992
epoch: 09, loss: -0.83563
epoch: 10, loss: -0.82270
epoch: 11, loss: -0.79446
epoch: 12, loss: -0.77873
epoch: 13, loss: -0.79728
epoch: 14, loss: -0.81098
epoch: 15, loss: -0.79840
epoch: 16, loss: -0.81007
epoch: 17, loss: -0.83928
